In [ ]:
from dotenv import load_dotenv
import os
import requests
from pathlib import Path
from typing import TypedDict, Optional
import math


class BookNode(TypedDict):
    isbn: str
    title: str
    group: str
    salesrank: Optional[int]
    log_salesrank: Optional[float]
    avg_rating: Optional[float]
    total_reviews: int
    log_total_reviews: Optional[float]
    categories: list[str]
    category_depth: int
    normalized_category_depth: float


class BookNodeContainer(TypedDict):
    nodes: list[BookNode]


project_root = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
dotenv_path = project_root / ".env"

load_dotenv(dotenv_path)
api_key = os.getenv("GCP_BOOKS_API")

if not api_key:
    raise ValueError("Missing GCP_BOOKS_API key. Check your .env file.")


def _fetch_book_node(isbn: str, api_key: str) -> Optional[BookNode]:
    """Fetch metadata for a single ISBN and return it as a BookNode dict."""
    try:
        base_url = "https://www.googleapis.com/books/v1/volumes"
        params = {"q": f"isbn:{isbn}", "key": api_key}

        response = requests.get(base_url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        if "items" not in data or not data["items"]:
            print(f"No book found for ISBN: {isbn}")
            return None

        book = data["items"][0]["volumeInfo"]

        # Build full title (title + subtitle + description)
        parts = [book.get("title", ""), book.get("subtitle", ""), book.get("description", "")]
        full_title = " — ".join([p.strip() for p in parts if p])

        # Extract key fields
        avg_rating = book.get("averageRating")
        total_reviews = book.get("ratingsCount") or 0
        categories = book.get("categories", []) or ["Unknown"]

        # Derived metrics
        log_total_reviews = round(math.log10(total_reviews), 2) if total_reviews > 0 else 0.0
        salesrank = None  # Not provided by Google Books, TODO: Fetch salesrank from Amazon API.
        log_salesrank = round(math.log10(salesrank), 2) if salesrank else None
        category_depth = len(categories)
        normalized_category_depth = round(min(category_depth / 8, 1.0), 2)

        return {
            "isbn": isbn,
            "title": full_title,
            "group": "Book",
            "salesrank": salesrank,
            "log_salesrank": log_salesrank,
            "avg_rating": avg_rating,
            "total_reviews": total_reviews,
            "log_total_reviews": log_total_reviews,
            "categories": categories,
            "category_depth": category_depth,
            "normalized_category_depth": normalized_category_depth,
        }

    except requests.RequestException as e:
        print(f"Error fetching ISBN {isbn}: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error for ISBN {isbn}: {e}")
        return None


def get_book_nodes(isbn_list: list[str], api_key: str) -> BookNodeContainer:
    """
    Accepts a list of ISBNs (one or many) and returns a single BookNodeContainer.
    Any book that fails to fetch is skipped.
    """
    nodes: list[BookNode] = []

    for isbn in isbn_list:
        node = _fetch_book_node(isbn, api_key)
        if node:
            nodes.append(node)
        else:
            print(f"Skipped ISBN: {isbn}")

    print(f"\nSuccessfully fetched {len(nodes)} of {len(isbn_list)} books.")
    return {"nodes": nodes}


if __name__ == "__main__":
    isbn_list = [
        "9780735211292",  # Atomic Habits
        "9780143127741",  # Educated
        "INVALID_ISBN",   # This will fail gracefully
        "9780062316097"   # Sapiens
    ]

    book_data = get_book_nodes(isbn_list, api_key) # This is the result to continue with

    import json
    print(json.dumps(book_data, indent=2, ensure_ascii=False))


No book found for ISBN: INVALID_ISBN
Skipped ISBN: INVALID_ISBN

Successfully fetched 3 of 4 books.
{
  "nodes": [
    {
      "isbn": "9780735211292",
      "title": "Atomic Habits — An Easy & Proven Way to Build Good Habits & Break Bad Ones — The #1 New York Times bestseller. Over 25 million copies sold! Translated into 60+ languages! Tiny Changes, Remarkable Results No matter your goals, Atomic Habits offers a proven framework for improving--every day. James Clear, one of the world's leading experts on habit formation, reveals practical strategies that will teach you exactly how to form good habits, break bad ones, and master the tiny behaviors that lead to remarkable results. If you're having trouble changing your habits, the problem isn't you. The problem is your system. Bad habits repeat themselves again and again not because you don't want to change, but because you have the wrong system for change. You do not rise to the level of your goals. You fall to the level of your system